<a href="https://colab.research.google.com/github/saturnMars/FM_2025/blob/main/Lab4_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent playground

In [ ]:
!pip install -U langchain langchain_openai langchain_huggingface langchain_community langchain_tavily

# Initialize the base language model

In [ ]:
local_inference = False

### A) Cloud inference
1. via [*Hugging Face’s Inference Providers*](https://huggingface.co/docs/inference-providers/en/index)
    - Create an account for the Hugging Face platform: [huggingface.co/join](https://huggingface.co/join)
    - Get the API key from dashboard: [huggingface.co/docs/hub/en/security-tokens](https://huggingface.co/docs/hub/en/security-tokens)
2. via [*OpenAI API*](https://auth.openai.com)
    - Create a new OpenAI account (free credits for new accounts): [auth.openai.com/create-account](https://auth.openai.com/create-account)
    - Generate the API key from the dashboard: [platform.openai.com/api-keys](https://platform.openai.com/api-keys)
3. via the [OpenRouter platform](https://openrouter.ai/)
    - Create an account: [openrouter.ai](https://openrouter.ai/)
    - Create a new API key: [openrouter.ai/settings/keys](https://openrouter.ai/settings/keys)

In [ ]:
from os import environ
environ["HF_TOKEN"] = ""
environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain_openai import ChatOpenAI

# Cloud inference via OpenAI
if not local_inference and environ.get('OPENAI_API_KEY'):
    llm_model = ChatOpenAI(model="gpt-5-nano", api_key=environ["OPENAI_API_KEY"])

    print(f"Cloud inference: model: \"{llm_model.model_name}\"")

# Cloud inference via HuggingFace
elif not local_inference and environ.get('HF_TOKEN'):
    llm_model = ChatOpenAI(
        base_url="https://router.huggingface.co/v1",
        model="Qwen/Qwen3-Next-80B-A3B-Instruct", # (1) Qwen/Qwen3-Next-80B-A3B-Instruct || (2) openai/gpt-oss-120b (3) CohereLabs/c4ai-command-r7b-12-2024
        api_key=environ["HF_TOKEN"])
    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")
else:
    llm_model = ChatOpenAI(
        base_url="https://openrouter.ai/api/v1",
        model="nvidia/nemotron-nano-9b-v2:free", # (1) mistralai/mistral-small-3.2-24b-instruct:free (1) nvidia/nemotron-nano-9b-v2:free
        api_key="")

    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")

Cloud inference: model: "gpt-5-nano"


### B) Local inference

In [ ]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

if local_inference:
    llm_model = ChatHuggingFace(
        llm = HuggingFacePipeline.from_model_id(
            model_id="allenai/OLMo-2-0425-1B-Instruct", #  allenai/OLMo-2-0425-1B-Instruct | Qwen/Qwen3-4B-Instruct-2507
            task ="text-generation",
            pipeline_kwargs={'dtype':"bfloat16"}
        ))
    print(f"Local Inference: \"{llm_model.llm.model_id}\"")

# Get the API token
1. Create an account on [Tavily platform](https://app.tavily.com/home)
2. Create a new API key: [https://app.tavily.com/home](https://app.tavily.com/home)


# Initialize the tool
This code demonstrates how to use the `TavilySearch` tool from the `langchain_tavily` package to perform a web search within a LangChain workflow.
2. It imports the TavilySearch class, which is a tool designed to query the Tavily Search API and return structured search results, such as URLs, snippets, and optionally images or answers.
3. The invoke method is then called with the query `"What is Italy’s current public debt?"`.
    - This method sends the query to the Tavily API and returns the search results as a dictionary containing information such as the original query, a list of result items (with titles, URLs, and content snippets), and possibly other metadata.
4. The results are printed to the output pane, allowing you to inspect the returned data.

The code also shows how to organize tools for later use by placing the search tool into a list called tools. This is useful when building more complex agent workflows that may use multiple tools for different tasks.

In [ ]:
from langchain_tavily import TavilySearch
from json import dumps

# Initialize the Tavily Search tool
search_tool = TavilySearch(max_results=2, tavily_api_key = "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")

# Try out the search tool
search_results = search_tool.invoke(input = "What is Italy’s current public debt?")
print(dumps(search_results, indent=4, ensure_ascii = False))

{
    "query": "What is Italy’s current public debt?",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "url": "https://en.wikipedia.org/wiki/Italian_government_debt",
            "title": "Italian government debt - Wikipedia",
            "content": "As of January 2014, the Italian government debt stands at €2.1 trillion (131.1% of GDP). Italy has the lowest share of public debt held by non-residents of all",
            "score": 0.91156644,
            "raw_content": null
        },
        {
            "url": "https://www.reuters.com/markets/europe/italys-public-debt-tops-3-trillion-euros-highest-record-2025-01-15/",
            "title": "Italy's public debt tops 3 trillion euros, highest on record | Reuters",
            "content": "Italy's debt climbed to 3,005.2 billion euros in November, compared with 2,981.3 billion euros in the previous month, Bank of Italy data showed.",
            "score": 0.890761,
            "

# Invoke the agent with a user query


In [ ]:
query = "What's the weather like today in Trento, Italy?"

### A) without the search tool

In [ ]:
from langchain.agents import create_agent
agent_executor = create_agent(
    model = llm_model,
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================

I can look that up for you. Do you want me to fetch the latest current conditions for Trento, Italy now? If yes, I’ll include:
- current temperature and weather (e.g., sunny, cloudy, rain)
- precipitation chances
- wind
- today’s high/low
- a brief forecast for the rest of the day (in °C)

If you have a preferred weather source (e.g., Meteo.it, The Weather Channel, AccuWeather), tell me and I’ll use that.


### B) with the search tool
![image.png](https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/agent.png?w=840&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=bd932835b919f5e58be77221b6d0f194)

In [ ]:
agent_executor = create_agent(
    model = llm_model,
    tools = [search_tool],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_EUPkc6gceLqkT7LGvpqp8jfV)
 Call ID: call_EUPkc6gceLqkT7LGvpqp8jfV
  Args:
    query: Trento weather today
    time_range: None
    start_date: None
    end_date: None
    include_domains: None
    exclude_domains: None
    search_depth: advanced
    include_images: False
    include_favicon: False
    topic: general
  tavily_search (call_iUMizfHhJupcFm5VCi8AwQp2)
 Call ID: call_iUMizfHhJupcFm5VCi8AwQp2
  Args:
    query: Trento meteo oggi
    time_range: None
    start_date: None
    end_date: None
    include_domains: ['meteo.it', 'ilmeteo.it', 'weather.com', 'yr.no', 'accuweather.com']
    exclude_domains: None
    search_depth: advanced
    include_images: False
    include_favicon: False
    topic: general
================================= T

# Create our custom tools

In [ ]:
def get_exam_score(exam_name: str) -> dict:
    """Get the expected score for a given exam."""

    # For demonstration purposes, we assume a perfect score (we have high expectations!)
    student_score = 30

    return {
        'exam_name': exam_name,
        'range': (0, 30),
        'score': student_score}

def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""

    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18

    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score,
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False # sorry :/
    }

    return results

In [ ]:
agent_executor = create_agent(
    model = llm_model,
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
query = 'Will I ever pass the FM 2025 exam?'

In [ ]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Will I ever pass the FM 2025 exam?
================================== Ai Message ==================================
Tool Calls:
  get_exam_score (call_OxhfVbUv0jUPAUQ9M7FI3vWv)
 Call ID: call_OxhfVbUv0jUPAUQ9M7FI3vWv
  Args:
    exam_name: FM 2025
================================= Tool Message =================================
Name: get_exam_score

{"exam_name": "FM 2025", "range": [0, 30], "score": 30}
================================== Ai Message ==================================
Tool Calls:
  parse_result (call_VaeAUs8S851bC8G1rxm1ylMx)
 Call ID: call_VaeAUs8S851bC8G1rxm1ylMx
  Args:
    score: 30
================================= Tool Message =================================
Name: parse_result

{"score": 30, "pass_threshold": 18, "passed": true, "cum_laude": false}
================================== Ai Message ==================================

Yes. Based on the data, you have passed the FM 2025 ex

In [ ]:
# Turnaround to avoid API limitation (rate-limited upstream)
from time import sleep
sleep(5)

# Human in the loop

In [ ]:
def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""

    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18

    # Ask for human approval if the score is passing
    accepted = None
    if score >= pass_threshold:
        user_input = input(f"Do you accept a score equal to {score} (yes/no): ").strip().lower()
        accepted = True if user_input == 'yes' else False

    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score,
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False, # sorry :/
        'acceptedByStudent': accepted
    }

    return results

In [ ]:
agent_executor = create_agent(
    model = llm_model,
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}}, config = {"configurable": {"thread_id": "101"}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

Do you accept a score equal to 30 (yes/no): no
================================ Human Message =================================

Will I ever pass the FM 2025 exam?
================================== Ai Message ==================================
Tool Calls:
  get_exam_score (call_cUtFCfE2CeboDPqas6MvbLme)
 Call ID: call_cUtFCfE2CeboDPqas6MvbLme
  Args:
    exam_name: FM 2025
================================= Tool Message =================================
Name: get_exam_score

{"exam_name": "FM 2025", "range": [0, 30], "score": 30}
================================== Ai Message ==================================
Tool Calls:
  parse_result (call_wv31camrzcVjG1RQaceFBWPJ)
 Call ID: call_wv31camrzcVjG1RQaceFBWPJ
  Args:
    score: 30
================================= Tool Message =================================
Name: parse_result

{"score": 30, "pass_threshold": 18, "passed": true, "cum_laude": false, "acceptedByStudent": false}
================================== Ai Message ===============

# Conversational agents (i.e., ChatBots)

In [ ]:
!pip install gradio

In [ ]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables import RunnableMap

In [ ]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Retrieve or create chat history for a session."""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Gradio

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [ ]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

# Chain the prompt with the LLM
chain = prompt | llm_model

# Create the chatbot with history support
chatbot = RunnableWithMessageHistory(chain, get_session_history=get_session_history, input_messages_key="input", history_messages_key="history")

In [ ]:
import gradio as gr
import re

def get_response(user_message, history):

    # Execute the model
    response = chatbot.invoke(input = {"input": user_message}, config={"configurable": {"session_id": session_id}})

    # Get the text
    generated_text = response.content

    # Clear text
    matches = list(re.finditer(user_message, generated_text))
    if matches:
        last_match = matches[-1]
        generated_text = generated_text[last_match.end() + 1 :]

    return generated_text.strip()

# Define the interface
session_id = 'user1'
interface = gr.ChatInterface(fn=get_response, type="messages", title=f"ChatBot", examples=["Tell me a joke", "What's the capital of France?", "What is the population of Trento?"])

# Launch the interface
interface.launch(share = True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ba437e1ebe4108489c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ba437e1ebe4108489c.gradio.live
